# Experiment
- Hybrid search
- Doc split

Reference: https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble

# Config

In [1]:
WEIGHTS = [0.5, 0.5]
K = 1
INDEX = "labse-major"
# INDEX = "gemini-major"

MODEL = "sentence-transformers/LaBSE"
# MODEL = "models/embedding-001"

DIST = "EUCLIDEAN_DISTANCE"

META = {
    "model": MODEL,
    "doc_size": 460,
    "doc_overlap": 20,
    "bm25-weight": WEIGHTS[0],
    "elastic-weight": WEIGHTS[1],
    "index": INDEX,
    "top k": K,
    "distance": DIST, 
    "technique": f"{MODEL} + Hybrid"
}

# My code

In [2]:
# import
import sys
sys.path.append("../")

from src.rag.hybrid_rag import HybridGeminiRAG
from src.service.provider import ProviderService
services = ProviderService()

major = services.get_categories().major
rag = HybridGeminiRAG(
    provider=services, 
    rag_config=services.get_categories().major, 
    update_notification_func=lambda x: x)

ensemble_retriever = rag.ensemble_retriever

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


From (44, 5) to 44
From (11, 4) to 11


# Test

In [3]:
from src.tests.retriever_eval import RetrieverEvaluation
eval = RetrieverEvaluation(root_path="../data", test_folder="test_major", save_path="./results")

!ls ../data/test_major

config.txt  hard_private_test_case.csv	public_test_case.csv
docs.csv    private_test_case.csv	sample_test_case.csv


# Labse

In [4]:
eval.eval_sample("sample_hybrid_docsplit.txt", ensemble_retriever, META)

In [4]:
eval.eval_public("labse_major_hybrid_public_docsplit.txt", ensemble_retriever, META)

In [4]:
eval.eval_private_hard("labse_major_hybrid_hard_private_docsplit.txt", ensemble_retriever, META)

# BM25

In [4]:
bm25 = rag.bm25

In [5]:
eval.eval_sample("sample_bm25_docsplit.txt", bm25, META)

In [6]:
eval.eval_public("public_bm25_docsplit.txt", bm25, META)

In [7]:
eval.eval_private_hard("private_bm25_docsplit.txt", bm25, META)

# Gemini

In [6]:
eval.eval_public("gemini_major_hybrid_public_docsplit.txt", ensemble_retriever, META)

In [7]:
eval.eval_private_hard("gemini_major_hybrid_hard_private_docsplit.txt", ensemble_retriever, META)